In [1]:
# setup
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
from IPython.display import HTML, display

data_path = Path("data")
data_csv = Path("data", "python_dev_universe.csv")
data_csv_gz = Path(data_path, "python_dev_universe.csv.gz")
data_parquet = Path("data", "python_dev_universe.parquet")


<html>
<head>
</head>
<body style="background-color: #FFFFFF;">
  <h1 align="center" style="font-weight: bold; font-style: italic; font-size: 390%;">Better Together - Pandas + Polars</h1>
  <table border="0" align="center" width="100%" bgcolor="#FFFFFF">
    <tr>
      <td align="center" width="50%" bgcolor="#FFFFFF">
        <img src="images/pandas_logo.1280x517.png" width="620" height="250">
      </td>
      <td align="center" width="50%" bgcolor="#FFFFFF">
        <img src="images/polars.round.400x400.png" width="250" height="250">
      </td>
    </tr>
    <tr>
      <td colspan="2" align="center" bgcolor="#FFFFFF">
        <img src="images/arrow-logo_horizontal.1800x936.png" width="481" height="250">
      </td>
    </tr>
  </table>
  <br>
  <div style="font-size: 300%;">
    <b>Better Together: Unleashing the Synergy of Pandas, Polars, and Apache Arrow</b><br>
    <b>Speaker:  Chris Brousseau</b> <br>
    <b>30 Nov 2023</b> 
  </div>
</body>
</html>
 </div>
</body>
</html>

<table class="custom-slide-table">
    <tr>
        <td class="image-content">
            <img src="./images/intro_slide_full.jpg" alt="Images of me - full slide">
        </td>
    </tr>
</table>

<table style="width: 100%; border-collapse: collapse; margin-top: 0;">
    <tr>
        <td style="vertical-align: top;">
            <h1 style="font-size: 48px; color: #333; margin-top: 0; margin-bottom: 10px;">TLDR</h1>
        </td>
    </tr>
    <tr>
        <td style="vertical-align: top;">
            <h2 style="font-size: 24px; color: #555; margin-top: 0; margin-bottom: 10px;">- Use Pandas - power (completeness) / flexibility / stability</h2>
        </td>
    </tr>
    <tr>
        <td style="vertical-align: top;">
            <h2 style="font-size: 24px; color: #555; margin-top: 0; margin-bottom: 10px;">- Add Polars where you can for speed / smaller less stable API</h2>
        </td>
    </tr>
    <tr>
        <td style="vertical-align: top;">
            <h2 style="font-size: 24px; color: #555; margin-top: 0; margin-bottom: 10px;">- Both getting faster - Arrow is the driver</h2>
        </td>
    </tr>
    <tr>
        <td style="vertical-align: top;">
            <h2 style="font-size: 24px; color: #555; margin-top: 0;">- Build better pipelines -- use together where it makes sense</h2>
        </td>
    </tr>
</table>


<h1>What is Apache Arrow / Why is it a Game Changer?</h1>

<p style="font-size: 24px; line-height: 2;">
  <strong>In-memory data format for tabular data</strong><br><br>
  <strong>Interoperability ==> Easier</strong> ...Arrow makes data program independent<br>
  <strong>Datatypes ==> More + Better</strong>  ...Strings; Missing data support<br>
  <strong>Speed ==> Faster</strong> ...Columnar format; zero-copy reads ==> transfer pointers + metadata<br>
</p>

<table border="1" style="width: 95%">
  <thead>
    <tr style="font-weight: bold;">
      <th style="text-align: center; padding: 10px">
        <img src="./images/arrow_simd.948x651.png" alt="in-memory columnar format" style="margin-right: 10px;">
      </th>
      <th style="text-align: center; padding: 10px">
        <img src="./images/arrow_copy.574x318.png" alt="zero-copy reads" style="margin-left: 10px;">
      </th>
    </tr>
  </thead>
  <tbody>
    <tr style="background-color: #FFFFFF; ; font-size: 24px;">
      <td colspan="3" style="font-weight: bold; text-align: center;">
        Image source: Apache Foundation
      </td>
    </tr>
    <tr>
      <td style="text-align: center; font-size: 18px;">
        <a href="https://arrow.apache.org/overview/">Apache Arrow Overview</a>
      </td>
      <td style="text-align: center; font-size: 18px;">
        <a href="https://arrow.apache.org/docs/format/Columnar.html">Columnar Format</a>
      </td>
      <td style="text-align: center; font-size: 18px;">
        <a href="https://pandas.pydata.org/docs/user_guide/pyarrow.html">Pandas PyArrow</a>
      </td>
    </tr>
  </tbody>
</table>


<h1>Key Differences - Packages</h1>

<table border="1" style="width: 80%; font-size: 24px;">
  <thead>
    <tr style="font-weight: bold;">
      <th style="vertical-align: bottom;">Feature</th>
      <th style="text-align: left;">
        <img src="./images/pandas_secondary.svg" alt="Pandas" style="width: 300px; max-width: 100%;">
      </th>
      <th style="text-align: left;">
        <img src="./images/polars.round.400x400.png" alt="Polars" style="width: 200px; max-width: 100%;">
      </th>
    </tr>
  </thead>
  <tbody>
    <tr style="height:100px"; style="background-color: #FFFFFF;">
      <td>First Release Date</td>
      <td>2008</td>
      <td>2019</td>
    </tr>
    <tr style="height:100px"; style="background-color: #F0F0F0;">
      <td>Current Release</td>
      <td>2.1.3</td>
      <td>0.19.13</td>
    </tr>
    <tr style="height:100px"; style="background-color: #FFFFFF;">
      <td>Programming Language</td>
      <td>C, Cython, Python</td>
      <td>Rust</td>
    </tr>
    <tr style="height:100px"; style="background-color: #FFFFFF;">
      <td>Project Goal</td>
      <td>To be the fundamental building block for python data analysis & manipulation - most powerful and flexible dataframe tool</td>
      <td>To provide lightning-fast dataframes that use all local resources</td>
    </tr>
  </tbody>
</table>


<h1>Key Differences - Memory</h1>

<table border="1" style="width: 80%; font-size: 24px;">
  <thead>
    <tr style="font-weight: bold;">
      <th style="vertical-align: bottom;">Feature</th>
      <th style="text-align: left;">
        <img src="./images/pandas_secondary.svg" alt="Pandas" style="width: 300px; max-width: 100%;">
      </th>
      <th style="text-align: left;">
        <img src="./images/polars.round.400x400.png" alt="Polars" style="width: 200px; max-width: 100%;">
      </th>
    </tr>
  </thead>
  <tbody>
    <tr style="height:100px"; style="background-color: #F0F0F0; font-weight: bold">
      <td><strong>Memory</strong></td>
      <td></td>
      <td></td>
    </tr>
    <tr style="height:100px"; style="background-color: #FFFFFF;">
      <td>Memory Backend</td>
      <td>Numpy (default) or <strong>Apache Arrow</strong></td>
      <td><strong>Apache Arrow</strong></td>
    </tr>
    <tr style="height:100px"; style="background-color: #F0F0F0;">
      <td>Memory implementation</td>
      <td>Pyarrow <br>(C++ wrapper on data)</td>
      <td>Arrow2 <br>(Rust wrapper on data)</td>
    </tr>
    <tr style="height:100px"; style="background-color: #FFFFFF;">
      <td>Larger-than-Memory/ Out-of-Core</td>
      <td>No  (but via Dask)</td>
      <td><strong>Native on Lazy df only (`collect(streaming=True`);  chunking + spill</strong></td>
    </tr>
    <tr style="height:100px"; style="background-color: #F0F0F0;">
      <td>Represent Missing Data</td>
      <td>"NaN" or "None"</td>
      <td>"null"</td>
    </tr>
  </tbody>
</table>


<h1>Key Differences - API</h1>

<table border="1" style="width: 80%; font-size: 24px;">
  <thead>
    <tr style="font-weight: bold;">
      <th style="vertical-align: bottom;">Feature</th>
      <th style="text-align: left;">
        <img src="./images/pandas_secondary.svg" alt="Pandas" style="width: 300px; max-width: 100%;">
      </th>
      <th style="text-align: left;">
        <img src="./images/polars.round.400x400.png" alt="Polars" style="width: 200px; max-width: 100%;">
      </th>
    </tr>
  </thead>
  <tbody>
    <tr style="height:70px"; style="background-color: #FFFFFF;">
      <td>Number of Methods</td>
      <td><strong><i>Many</i></strong></td>
      <td><strong><i>not 1:1</i></strong></td>
    </tr>
    <tr style="height:70px"; style="background-color: #F0F0F0;">
      <td>Index/Multindex</td>
      <td>Yes</td>
      <td><strong>No - "index free"</strong></td>
    </tr>
    <tr style="height:70px"; style="background-color: #FFFFFF;">
      <td>Nullable dtype</td>
      <td>Yes</td>
      <td>Yes</td>
    </tr>
    <tr style="height:70px"; style="background-color: #F0F0F0;">
      <td>API mode</td>
      <td>Eager</td>
      <td>Eager (+ Lazy)</td>
    </tr>
    <tr style="height:70px"; style="background-color: #FFFFFF;">
      <td>Query Optimization</td>
      <td>No</td>
      <td>Yes (with Lazy)</td>
    </tr>
    <tr style="height:100px"; style="background-color: #F0F0F0;">
      <td>Parallelization</td>
      <td>No - single threaded</td>
      <td><strong>Yes - multithreaded</td>
    </tr>
    <tr style="height:70px"; style="background-color: #FFFFFF;">
      <td>SIMD</td>
      <td>No</td>
      <td><strong>Yes</td>
    </tr>
  </tbody>
</table>


<h1>Recent Updates</h1>
<table border="1" style="width: 80%; font-size: 24px;">
  <thead>
    <tr style="font-weight: bold;">
      <th style="vertical-align: bottom;">2023 - Speed & Consistency</th>
      <th style="text-align: left;">
        <img src="./images/pandas_secondary.svg" alt="Pandas" style="width: 300px; max-width: 100%;">
      </th>
      <th style="text-align: left;">
        <img src="./images/polars.round.400x400.png" alt="Polars" style="width: 200px; max-width: 100%;">
      </th>
    </tr>
  </thead>
  <tbody>
    <tr style="background-color: #FFFFFF;">
      <td>Backend</td>
      <td>Apache Arrow. <strong>Pyarrow required after 3.0.0</strong></td>
      <td>+cloud reading; +speed +bugfix</td>
    </tr>
    <tr style="height:100px"; style="background-color: #F0F0F0;">
      <td></td>
      <td><i>Lazy </i><strong>Copy-on-Write</strong><br>simplifies API<br>only mod one object</td>
      <td></td>
    </tr>
    <tr style="height:100px"; style="background-color: #FFFFFF;">
      <td>Feature Flags / optional dependencies</td>
      <td>pip install <br>pandas[aws, performance]</td>
      <td>pip install <br>polars[pandas, ffspec]</td>
    </tr>
    <tr style="height:100px"; style="background-color: #F0F0F0;">
      <td>Reference</td>
      <td><a href="https://pandas.pydata.org/docs/whatsnew/index.html">pandas release notes</td>
      <td><a href="https://github.com/pola-rs/polars/releases">polars release notes</td>
    </tr>
  </tbody>
</table>

<h1>But how much faster?</h1>
<table border="1" style="width: 80%; ">
  <thead>
    <tr style="font-size: 24px; font-weight: bold;">
      <th style="vertical-align: bottom;">Anecdotes & Considerations</th>
      <th style="text-align: left;">
        <img src="./images/pandas_secondary.svg" alt="Pandas" style="width: 300px; max-width: 100%;">
      </th>
      <th style="text-align: left;">
        <img src="./images/polars.round.400x400.png" alt="Polars" style="width: 200px; max-width: 100%;">
      </th>
    </tr>
  </thead>
  <tbody>
    <tr style="height:70px; font-size: 24px; background-color: #FFFFFF;">
      <td>read csv +csv.gz</td>
      <td></td>
      <td><strong>~3x-20x faster</strong></td>
    </tr>
    <tr style="height:70px; font-size: 24px; background-color: #F0F0F0;">
      <td>read parquet</td>
      <td></td>
      <td>~1x-5x</td>
    </tr>
    <tr style="height:70px; font-size: 24px; background-color: #FFFFFF;">
      <td>groupby</td>
      <td></td>
      <td>~10x</td>
    </tr>
    <tr style="height:70px;  font-size: 24px; background-color: #F0F0F0;">
      <td>Other Considerations</td>
      <td>Huge Ecosystem / examples</td>
      <td>Smaller but growing Ecosystem</td>
    <tr style="height:100px; font-size: 24px; background-color: #FFFFFF;">
      <td>API</td>
      <td><strong>Stable</strong></td>
      <td><strong>Less Stable - but improving</strong></td>
    </tr>
      <tr style="height:70px; font-size: 24px; background-color: #F0F0F0;">
      <td> </td>
      <td> </td>
      <td>Categorical; some window functions / plotting /etc.</td>
    </tr>
    <tr style="height:70px; font-size: 24px; background-color: #FFFFFF;">
      <td><strong>Reference - see repo</strong></td>
      <td><a href="https://pandas.pydata.org/docs/reference/index.html">pandas api</td>
      <td><a href="https://pola-rs.github.io/polars/py-polars/html/reference/">polars api</td>
    </tr>
  </tbody>
</table>

<style>
    .custom-slide-table {
        width: 100%;
        table-layout: fixed;
    }

    .custom-slide-table td {
        vertical-align: middle;
    }

    .custom-title {
        font-size: 32px;
        text-align: left;
        margin-top: 20px;
        margin-bottom: 20px;
    }
</style>

<h1 class="custom-title">Caution when comparing baselines!  Configs have a big impact</h1>

<table class="custom-slide-table">
    <tr>
        <td class="image-content">
            <img src="./images/csv_read_speed_comparison.jpg" alt="A lot faster!">
        </td>
    </tr>
</table>


# Get the Best out of Pandas

### 1- use pyarrow for I/O & nullable dtypes (faster)
pd.read_csv( my_datat.csv",  engine="pyarrow" dtype_backend="pyarrow")<br>pd.read_parquet) <br>
<br>
### 2- set pyarrow for all string data (faster/smaller)
pd.options.future.infer_string = True   <br>
<br>
### 3- enable Copy-On-Write (lazy + more consistent api) -- just do this anyway; will be default!
pd.options.mode.copy_on_write = True <br>
<br>

# some syntax differences

# Use Columnar Example


# Use Lazy Example

In [2]:
# polars Eager API
df_pl2 = pl.read_parquet(data_parquet).filter(
    (pl.col("occupation") == "Data Engineer")
    & (pl.col("psf_membership_status") != "Not Yet a Member")
)

In [3]:
# polars Lazy API - chain transformations then collect()

df_pl2 = (
    pl.read_parquet(data_parquet)
    .lazy()
    .filter(
        (pl.col("occupation") == "Apache Arrow Understudy")
        & (pl.col("psf_membership_status") == "Contributing")
    )
    .collect()
)

<h1 style="font-size: 32px; color: #333; margin-bottom: 20px;">Integrating Pandas and Polars</h1>

<table style="width: 90%;">
    <tr>
        <td style="vertical-align: top; padding-right: 10px;">
            <h2 style="font-size: 32px; font-weight: bold; color: #555; margin: 15px 0;">Why</h2>
            <ul>
                <li style="font-size: 24px; color: #777; margin: 10px 0;">Creating new pipeline (great!)</li>
                <li style="font-size: 24px; color: #777; margin: 10px 0;">Need big speed improvements + worth your time + budget</li>
                <li style="font-size: 24px; color: #777; margin: 10px 0;">Expensive Pandas ops with direct matches (e.g. select)</li>
                <li style="font-size: 24px; color: #777; margin: 10px 0;">Input data is Polars-friendly (a lot)</li>
                <ul style="font-size: 24px; margin-left: 40px; list-style-type: disc;">
                    <li style="font-size: 24px;"> Pandas df</li>
                    <li style="font-size: 24px;"> Flat files</li>
                    <li style="font-size: 24px;"> Standard db (sql) or supported datastore</li>
                    <li style="font-size: 24px;"> Already in Arrow (e.g. Spark)</li>
                    <li style="font-size: 24px;"> <strong> API does NOT require  pandas (e.g. Snowflake)</strong></li>
                </ul>
            </ul>
        </td>
        <td style="vertical-align: top; padding-left: 10px;">
            <h2 style="font-size:32px; font-weight: bold; color: #555; margin: 15px 0;">How</h2>
            <ul style="list-style-type: none;">
                <li style="font-size: 24px; color: #777; margin: 10px 0;">Use polars</li>
            </ul>
            <div style="font-size: 24px; margin-left: 40px;">
                <code>polars.from_pandas</code><br>
                <code>polars.to_pandas</code><br>
            </div>
            <ul style="list-style-type: none;">
                <li style="font-size: 24px; color: #777; margin: 10px 0;">Use arrow directly</li>
            </ul>
            <div style="font-size: 24px; margin-left: 40px;">
                <strong>Pandas</strong> <br>
                <code>import pyarrow as pa</code><br>
                <code>import pandas as pd</code><br>
                <code>table = pa.Table.from_pandas(df)</code><br>
                <code>df_new = table.to_pandas()</code><br><br>
                <strong>Polars</strong> <br>
                <code>polars.from_arrow</code><br>
                <code>polars.to_arrow</code><br>
            </div>
        </td>
    </tr>
</table>


<table style="width: 100%; border-collapse: collapse; margin-top: 0;">
    <tr>
        <td style="vertical-align: top;">
            <h1 style="font-size: 48px; color: #333; margin-top: 0; margin-bottom: 10px;">TLDR</h1>
        </td>
    </tr>
    <tr>
        <td style="vertical-align: top;">
            <h2 style="font-size: 24px; color: #555; margin-top: 0; margin-bottom: 10px;">- Use Pandas - power (completeness) / flexibility / stability</h2>
        </td>
    </tr>
    <tr>
        <td style="vertical-align: top;">
            <h2 style="font-size: 24px; color: #555; margin-top: 0; margin-bottom: 10px;">- Add Polars where you can for speed / smaller less stable API</h2>
        </td>
    </tr>
    <tr>
        <td style="vertical-align: top;">
            <h2 style="font-size: 24px; color: #555; margin-top: 0; margin-bottom: 10px;">- Both getting faster - Arrow is the driver</h2>
        </td>
    </tr>
    <tr>
        <td style="vertical-align: top;">
            <h2 style="font-size: 24px; color: #555; margin-top: 0;">- Build better pipelines -- use together where it makes sense</h2>
        </td>
    </tr>
</table>


<table style="border-collapse: collapse; background-color: #FFFFFF; width: 100%; font-size: 48px;"">
    <tr>
        <td style="padding: 8px; background-color: #FFFFFF; text-align: left; font-weight: bold;">Thank you!</td>
    </tr>
    <tr>
        <td style="padding: 8px; background-color: #FFFFFF; text-align: left">Chris Brousseau</td>
    </tr>
    <tr>
        <td style="padding: 8px; background-color: #FFFFFF;text-align: left; font-weight: normal;">chris@surfaceowl.com</td>
    </tr>
    <tr>
        <td style="padding: 8px; background-color: #FFFFFF;text-align: left; width: 50%; font-size: 24px; background-color: #FFFFFF;">
            <a href="https://github.com/surfaceowl/talk_pandas_polars_arrow" target="_blank"> GitHub Repo: https://github.com/surfaceowl/talk_pandas_polars_arrow</a>
        </td>
        <td style="padding: 8px; background-color: #FFFFFF; text-align: center;">
            <img src="images/pybay_talk_pandas_polars_qr_code.png" alt="QR Code for PyBay Talk" style="max-width: 100%; height: auto;">
        </td>
    </tr>
</table>


# Reference - Links

pandas
https://pandas.pydata.org/docs/whatsnew/index.html

polars
https://pola-rs.github.io/polars/

apache arrow
https://arrow.apache.org/overview/

pyarrrow
https://arrow.apache.org/docs/python/index.html

dataframe API standard
https://data-apis.org/dataframe-api/draft/index.html
https://ponder.io/how-the-python-dataframe-interchange-protocol-makes-life-better/

Arrow Revolution
https://datapythonista.me/blog/pandas-20-and-the-arrow-revolution-part-i

# interesting articles/video
https://www.youtube.com/watch?v=QfLzEp-yt_U  # Richey Vink; Polars Creator
https://kyleake.medium.com/pandas-to-polars-a-comprehensive-transition-guide-81b6f50e9154

# convert to/from pandas/pyarrow
https://arrow.apache.org/docs/python/pandas.html

# Reference - Datatypes
https://pandas.pydata.org/docs/user_guide/basics.html#dtypes
https://pola-rs.github.io/polars/py-polars/html/reference/datatypes.html
https://arrow.apache.org/docs/python/pandas.html
